In [ ]:
from django.test import TestCase, Client
from django.urls import reverse
from django.contrib.auth import get_user_model
from public.models import UsuarioPersonalizado, PerfilAluno, PerfilProfessor, Projeto

class TestViews(TestCase):
    def setUp(self):
        self.client = Client()
        self.User = get_user_model()
        
        # Criar usuário para testes
        self.usuario = self.User.objects.create_user(
            username='testuser',
            email='test@example.com',
            password='testpassword123'
        )
        
    def test_index_usuario_nao_autenticado(self):
        """Teste: usuário não autenticado deve ser redirecionado para página de login"""
        response = self.client.get(reverse('public:index'))
        self.assertEqual(response.status_code, 302)  # Verifica o redirecionamento
        # Verificar se o redirecionamento é para a página de login
        self.assertIn('/login/', response.url)  # Ajuste conforme o padrão de URL do seu sistema
        
    def test_login_view(self):
        """Teste: página de login deve ser acessível"""
        response = self.client.get(reverse('public:login'))
        self.assertEqual(response.status_code, 200)
        
    def test_login_autenticacao_sucesso(self):
        """Teste: login com credenciais válidas deve redirecionar para index"""
        response = self.client.post(reverse('public:login'), {
            'username': 'testuser',
            'password': 'testpassword123'
        })
        self.assertEqual(response.status_code, 302)
        self.assertEqual(response.url, reverse('public:index'))
        
    def test_login_autenticacao_falha(self):
        """Teste: login com credenciais inválidas deve retornar à página de login"""
        response = self.client.post(reverse('public:login'), {
            'username': 'testuser',
            'password': 'senhaerrada'
        })
        self.assertEqual(response.status_code, 200)
        
    def test_acesso_pagina_protegida_usuario_autenticado(self):
        """Teste: usuário autenticado deve acessar páginas protegidas"""
        self.client.login(username='testuser', password='testpassword123')
        response = self.client.get(reverse('public:index'))
        self.assertEqual(response.status_code, 200)
        
    def test_logout(self):
        """Teste: logout deve redirecionar para a página de login"""
        self.client.login(username='testuser', password='testpassword123')
        response = self.client.get(reverse('public:logout'))
        self.assertEqual(response.status_code, 302)
        self.assertIn('/login/', response.url)